In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, lower, regexp_replace
from nltk.corpus import stopwords

spark = SparkSession.builder \
    .appName("Movie Word Count with Preprocessing") \
    .getOrCreate()

df = spark.read.csv("C:/Users/quatt/movies.csv", header=True)

words_df = df.select(
    explode(
        split(
            regexp_replace(lower(col("Film")), "[^a-zA-Z0-9\s]", ""), 
            "\s+"  
        )
    ).alias("word")
)

stop_words = stopwords.words('english')
words_df = words_df.filter(~col("word").isin(stop_words))  

word_counts = words_df.groupBy("word").count()

most_common_word = word_counts.orderBy(col("count").desc()).first()

print(f"Самое частое слово - '{most_common_word['word']}' ({most_common_word['count']} раза)")

spark.stop()

Самое частое слово - 'love' (4 раза)
